In [1]:
# for Github
from PIL import Image
import os.path
import os
import numpy as np
import pandas as pd
from os import listdir
import array
import time
from chainer.datasets import cifar
from annoy import AnnoyIndex
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from os import listdir
fltr_list = [filename for filename in listdir() if not filename.startswith('.')]

In [3]:
width = 32
height = 32
rgb = 3
n_tree = 500
search_k = 500
metric = 'manhattan'
name = '-size'+str(width)+'x'+str(height)+'x3-tree'+str(n_tree)+'-searchK'+str(search_k)+'-metric_'+metric

In [4]:
def create_train(fileList):
    train = []
    
    labelList = np.array( pd.read_csv('image_category_lables.csv') )
    
    for fileName in fileList:
              
        label = "";
        for labelInfo in labelList: #labelInfo：[image_dir_id, category]
                
            if labelInfo[0] == fileName:
                label = labelInfo[1]
                break
        
        # imageをセット
        path = 'train_dataset/' + fileName
        images = os.listdir(path)
        for imageName in images:
        
            tmp = []
        
            #imageとlabelをセットで格納
            imageTuple = [] 
            # 画像を配列で読み込む
            image = Image.open('train_dataset/'+fileName+'/'+imageName) 
            
            # resize -> array
            image = image.resize((width, height)) 
            
            # 画像を一次元配列に変換
            imageFlatten = np.ravel(image)
            imageFlatten = imageFlatten.astype(np.float32)
            
            # 画像の正規化
#             imageFlatten = imageFlatten/256

            tmp.append(imageFlatten)
            tmp.append(label)
            tmp.append('./train_dataset/'+fileName+'/'+imageName)
            
            train.append(tmp)

    return train


from os import listdir
path = './train_dataset'
fileList = [filename for filename in listdir(path) if not filename.startswith('.')]

t1 = time.time()

train = create_train(fileList)

t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

経過時間：110.88942313194275


In [5]:
len(train)

46053

In [19]:
len(train[0][0])

3072

In [6]:
def create_test(fileList):
    test = []
    id_list = []
        
    for fileName in fileList:
            
        # imageをセット
        path = 'test_dataset/' + fileName
        images = os.listdir(path)
        for imageName in images:
            # 画像を配列で読み込む
            image = Image.open('test_dataset/'+fileName+'/'+imageName) 
                
            # resize -> array
            image = image.resize((width, height)) 
            
            # 画像を一次元配列に変換
            imageFlatten = np.ravel(image)
            imageFlatten = imageFlatten.astype(np.float32)
            
            # 画像の正規化
#             imageFlatten = imageFlatten/256
                                  
            test.append(imageFlatten)
            id_list.append(fileName)

    return test,id_list


from os import listdir
path = './test_dataset'
fileList = [filename for filename in listdir(path) if not filename.startswith('.')]

t1 = time.time()

test,id_list = create_test(fileList)

t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

経過時間：2.427475929260254


In [7]:
x = []
y = []
for i in range(len(train)):
    x.append(train[i][0])
    y.append(train[i][1])

In [8]:
dim = width * height * rgb
annoy_model = AnnoyIndex(dim, metric)

t1 = time.time()

for i in range(len(x)):
    imageVector = x[i]
    annoy_model.add_item(i, imageVector)
    
t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")


経過時間：22.132113933563232


In [9]:
t1 = time.time()

annoy_model.build(n_tree)

t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

経過時間：91.51725006103516


In [10]:
# annoy_model.save("similaritySearchModel.ann")

In [17]:
predict = []
ids = []

t1 = time.time()

for i in range(len(test)):
    imageVector = test[i]
    if i == 100 or i == 300 or i == 500:
        t2_1 = time.time()
        elapsed_time = t2_1-t1
        print(i)
        print(f"経過時間：{elapsed_time}")

    
    # trainの写真のインデックスを返している
    predict_index = annoy_model.get_nns_by_vector(imageVector, 10 , search_k=search_k )
    
    for j in range(len(predict_index)):
        # train[predict_index[j]] が類似画像なので，対応するラベルをsubmtにする
        
        predict_label = train[predict_index[j]][1]
                
        ids.append(id_list[i])
        predict.append(predict_label)
    
t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

100
経過時間：0.4679141044616699
300
経過時間：1.3610057830810547
---------------
300
id_00001197
./train_dataset/id_00007893/01_2_side.jpg
Shirts
---------------
---------------
300
id_00001197
./train_dataset/id_00002109/02_2_side.jpg
Shirts
---------------
---------------
300
id_00001197
./train_dataset/id_00000285/02_7_additional.jpg
Tees_Tanks
---------------
---------------
300
id_00001197
./train_dataset/id_00006987/04_3_back.jpg
Shirts
---------------
---------------
300
id_00001197
./train_dataset/id_00000238/01_1_front.jpg
Dresses
---------------
---------------
300
id_00001197
./train_dataset/id_00003082/04_1_front.jpg
Shirts
---------------
---------------
300
id_00001197
./train_dataset/id_00000285/02_4_full.jpg
Tees_Tanks
---------------
---------------
300
id_00001197
./train_dataset/id_00007052/02_2_side.jpg
Dresses
---------------
---------------
300
id_00001197
./train_dataset/id_00006479/01_2_side.jpg
Dresses
---------------
---------------
300
id_00001197
./train_dataset/id_0

In [13]:
submit = pd.DataFrame()
submit['id'] = ids
submit['pred'] = predict
submit.to_csv('result'+name+'.csv' ,header=False, index=False)